In [1]:
import fev

In [2]:
# Create a task from a dataset stored on Hugging Face Hub
task = fev.Task(
    dataset_path="autogluon/chronos_datasets",
    dataset_config="monash_kdd_cup_2018",
    horizon=12,
)

In [3]:
# Load data available as input to the forecasting model
past_data, future_data = task.get_input_data()

Validating dataset format (num_proc=10):   0%|          | 0/270 [00:00<?, ? examples/s]

In [4]:
# past data before the forecast horizon.
past_data

Dataset({
    features: ['id', 'timestamp', 'target', 'city', 'station', 'measurement'],
    num_rows: 270
})

In [5]:
past_data[0]

{'id': 'T000000',
 'timestamp': array(['2017-01-01T14:00:00.000', '2017-01-01T15:00:00.000',
        '2017-01-01T16:00:00.000', ..., '2018-03-31T01:00:00.000',
        '2018-03-31T02:00:00.000', '2018-03-31T03:00:00.000'],
       dtype='datetime64[ms]'),
 'target': array([453., 417., 395., ...,  99., 102.,  97.], dtype=float32),
 'city': 'Beijing',
 'station': 'aotizhongxin_aq',
 'measurement': 'PM2.5'}

In [6]:
# future data that is known at prediction time (item ID, future timestamps, and known covariates)
future_data

Dataset({
    features: ['id', 'timestamp', 'city', 'station', 'measurement'],
    num_rows: 270
})

In [7]:
future_data[0]

{'id': 'T000000',
 'timestamp': array(['2018-03-31T04:00:00.000', '2018-03-31T05:00:00.000',
        '2018-03-31T06:00:00.000', '2018-03-31T07:00:00.000',
        '2018-03-31T08:00:00.000', '2018-03-31T09:00:00.000',
        '2018-03-31T10:00:00.000', '2018-03-31T11:00:00.000',
        '2018-03-31T12:00:00.000', '2018-03-31T13:00:00.000',
        '2018-03-31T14:00:00.000', '2018-03-31T15:00:00.000'],
       dtype='datetime64[ms]'),
 'city': 'Beijing',
 'station': 'aotizhongxin_aq',
 'measurement': 'PM2.5'}

In [8]:
import numpy as np


# Make predictions
def naive_forecast(y: list, horizon: int) -> list:
    return [y[np.isfinite(y)][-1] for _ in range(horizon)]

predictions = []
for ts in past_data:
    predictions.append(
        {"predictions": naive_forecast(y=ts[task.target_column], horizon=task.horizon)}
    )

In [9]:
eval_summary = task.evaluation_summary(predictions, model_name="naive")
eval_summary

Casting the dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

{'model_name': 'naive',
 'dataset_name': 'chronos_datasets_monash_kdd_cup_2018',
 'dataset_fingerprint': 'e91f446bd0b05d16',
 'dataset_path': 'autogluon/chronos_datasets',
 'dataset_config': 'monash_kdd_cup_2018',
 'horizon': 12,
 'cutoff': -12,
 'lead_time': 1,
 'min_ts_length': 13,
 'max_context_length': None,
 'seasonality': 1,
 'eval_metric': 'MASE',
 'extra_metrics': [],
 'quantile_levels': None,
 'id_column': 'id',
 'timestamp_column': 'timestamp',
 'target_column': 'target',
 'multiple_target_columns': None,
 'past_dynamic_columns': [],
 'excluded_columns': [],
 'test_error': 3.3784518866750513,
 'training_time_s': None,
 'inference_time_s': None,
 'fev_version': '0.2.0',
 'MASE': 3.3784518866750513}